In [ ]:
import seaborn as sns
import pandas as pd
import glob
import os
from collections import defaultdict

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data_path = "/content/drive/MyDrive/VPN Deprecated/data"
out_path = "/content/drive/MyDrive/VPN Deprecated/Output"
STRONG_PROTOCOLS = {"wireguard", "shadowsocks", "nordwhisper", "openvpn", "xray"}

In [ ]:
def canon_protocol(p: str) -> str:
    """Normalize attribution tokens so variants collapse to canonical protocol names."""
    t = str(p).strip().lower()
    if not t:
        return ""

    # OpenVPN variants
    if "openvpn" in t or t in {"tcp", "udp", "openvpn2"}:
        return "openvpn"

    # WireGuard variants
    if "wireguard" in t or t == "wg":
        return "wireguard"

    # Shadowsocks variants
    if "shadowsocks" in t or t in {"ss2", "ss"}:
        return "shadowsocks"

    # NordWhisper variants
    if "nordwhisper" in t:
        return "nordwhisper"

    # Xray variants
    if t.startswith("xray") or t.startswith("x-ray"):
        return "xray"

    return t

def split_protocols_cell(protocols_cell):
    """Return set of canonicalized attribution protocols for a row."""
    if pd.isna(protocols_cell):
        return set()
    return {canon_protocol(p) for p in str(protocols_cell).split(",") if str(p).strip()}

def split_censys_cell(censys_cell):
    """Return set of uppercase Censys protocol labels for a row."""
    if pd.isna(censys_cell):
        return set()
    return {c.strip().upper() for c in str(censys_cell).split(",") if c.strip()}

def row_strong_protocol_set(protocols_cell, censys_protocols_cell):
    """
    Return the set of canonical STRONG protocols present in the row,
    counted ONCE per row (server), with rule:
      - If Censys reports OPENVPN for that IP, do NOT count OpenVPN as strong from attribution.
    """
    attr_set = split_protocols_cell(protocols_cell)
    if not attr_set:
        return set()

    censys_set = split_censys_cell(censys_protocols_cell)

    # Key rule: if Censys sees OPENVPN, don't treat attributed OpenVPN as "strong"
    if "OPENVPN" in censys_set:
        attr_set.discard("openvpn")

    return {p for p in attr_set if p in STRONG_PROTOCOLS}

def is_row_strong(protocols_cell, censys_protocols_cell) -> bool:
    """Row is included if it has ANY strong protocol after applying the Censys-aware rule."""
    return len(row_strong_protocol_set(protocols_cell, censys_protocols_cell)) > 0

def print_strong_counts_for_app(app: str, df_strong: pd.DataFrame):
    """
    Print per-app counts for strong protocols.
    IMPORTANT: counts are per-row/server (each protocol counted at most once per row).
    """
    counts = defaultdict(int)

    for _, row in df_strong.iterrows():
        prot_set = row_strong_protocol_set(row.get("protocols"), row.get("censys_protocols"))
        for p in prot_set:
            counts[p] += 1

    print(f"\n=== {app} ===")
    print(f"Strong rows (after Censys-aware filter): {len(df_strong)}")
    if not counts:
        print("  (no strong protocols)")
        return

    for k in sorted(counts.keys()):
        print(f"  {k}: {counts[k]}")

In [ ]:
folders = [p for p in glob.glob(data_path + "/*") if os.path.isdir(p)]
os.makedirs(out_path, exist_ok=True)

In [ ]:
arr_counts = []

for folder in folders:
    app = folder.split("/")[-1]
    file_csv = f"{folder}/{app}_attribution.csv"
    censys_csv = f"{out_path}/{app}/ip_vpn_protocols.csv"

    try:
        df_attr = pd.read_csv(file_csv)
        censys_df = pd.read_csv(censys_csv).rename(columns={"IP": "ip", "Protocols": "censys_protocols"})

        # Left join to keep all attribution rows even if Censys missing
        df = df_attr.merge(censys_df, on="ip", how="left")

        # Filter to strong rows (Censys-aware)
        df_strong = df[df.apply(lambda r: is_row_strong(r.get("protocols"), r.get("censys_protocols")), axis=1)].copy()

        # Print per-app strong counts (per-row/server)
        print_strong_counts_for_app(app, df_strong)

        # Build protocol counts per app (per-row/server, deduped per row)
        attributed_protocols = defaultdict(int)
        for _, row in df_strong.iterrows():
            prot_set = row_strong_protocol_set(row.get("protocols"), row.get("censys_protocols"))
            for p in prot_set:
                attributed_protocols[p] += 1

        for prot, cnt in attributed_protocols.items():
            arr_counts.append([app, prot, cnt])

    except Exception as e:
        print(f"Error processing {file_csv}: {e}")

df_counts = pd.DataFrame(arr_counts, columns=["app", "protocol", "count"])
df_counts.to_csv(f"{out_path}/attributed_protocols_count.csv", index=False)



=== germany.vpn ===
Strong rows (after Censys-aware filter): 114
  openvpn: 4
  shadowsocks: 56
  wireguard: 90

=== com.zoogvpn.android ===
Strong rows (after Censys-aware filter): 172
  openvpn: 172
  shadowsocks: 169
  wireguard: 170
  xray: 165

=== de.mobileconcepts.cyberghost ===
Strong rows (after Censys-aware filter): 1584
  openvpn: 291
  wireguard: 1584

=== com.wsandroid.suite ===
Strong rows (after Censys-aware filter): 4581
  wireguard: 4581

=== com.surfshark.vpnclient.android ===
Strong rows (after Censys-aware filter): 3465
  wireguard: 3465

=== com.nordvpn.android ===
Strong rows (after Censys-aware filter): 8838
  nordwhisper: 7304
  openvpn: 8838
  wireguard: 8731

=== com.vpn99 ===
Strong rows (after Censys-aware filter): 1232
  openvpn: 203
  shadowsocks: 596
  wireguard: 326
  xray: 433

=== com.ixolit.ipvanish ===
Strong rows (after Censys-aware filter): 3275
  wireguard: 3275

=== com.instabridge.android ===
Strong rows (after Censys-aware filter): 336
  shado

In [ ]:
arr_pairs = []

for folder in folders:
    app = folder.split("/")[-1]
    file_csv = f"{folder}/{app}_attribution.csv"
    censys_csv = f"{out_path}/{app}/ip_vpn_protocols.csv"

    try:
        df_attr = pd.read_csv(file_csv)
        censys_df = pd.read_csv(censys_csv).rename(columns={"IP": "ip", "Protocols": "censys_protocols"})

        # Left join retains all attribution rows, but pairs require Censys present
        df = df_attr.merge(censys_df, on="ip", how="left")

        # Filter to strong rows (Censys-aware)
        df_strong = df[df.apply(lambda r: is_row_strong(r.get("protocols"), r.get("censys_protocols")), axis=1)].copy()
        print(f"{app} (rows used for pairs): {len(df_strong)}")

        pairs = defaultdict(int)

        for _, row in df_strong.iterrows():
            protocols_cell = row.get("protocols")
            censys_cell = row.get("censys_protocols")

            if pd.isna(protocols_cell):
                continue
            if pd.isna(censys_cell):
                # no Censys match -> cannot form coexisting pairs
                continue

            censys_protocols = [c.strip().upper() for c in str(censys_cell).split(",") if c.strip()]
            censys_set = set(censys_protocols)

            # Use row-level set (deduped per row) so tcp+udp doesn't double count
            attr_set = split_protocols_cell(protocols_cell)

            # Apply key rule: if censys has OPENVPN, discard openvpn from attr side
            if "OPENVPN" in censys_set:
                attr_set.discard("openvpn")

            # Keep only strong protocols on attribution side
            attr_strong_set = {p for p in attr_set if p in STRONG_PROTOCOLS}
            if not attr_strong_set:
                continue

            for prot in attr_strong_set:
                for censys_protocol in censys_protocols:
                    pairs[(prot, censys_protocol)] += 1

        for (prot, censys_protocol), count in pairs.items():
            arr_pairs.append([app, prot, censys_protocol, count])

    except Exception as e:
        print(f"Error processing {file_csv}: {e}")

dual_df = pd.DataFrame(arr_pairs, columns=["app", "protocol", "censys_protocol", "count"])
dual_df.to_csv(f"{out_path}/dual_protocols.csv", index=False)

dual_df

germany.vpn (rows used for pairs): 114
com.zoogvpn.android (rows used for pairs): 172
de.mobileconcepts.cyberghost (rows used for pairs): 1584
com.wsandroid.suite (rows used for pairs): 4581
com.surfshark.vpnclient.android (rows used for pairs): 3465
com.nordvpn.android (rows used for pairs): 8838
com.vpn99 (rows used for pairs): 1232
com.ixolit.ipvanish (rows used for pairs): 3275
com.instabridge.android (rows used for pairs): 336
com.browsec.vpn (rows used for pairs): 22
ch.protonvpn.android (rows used for pairs): 1225
com.gaditek.purevpnics (rows used for pairs): 915
com.bitdefender.vpn (rows used for pairs): 2995


,app,protocol,censys_protocol,count
0,germany.vpn,shadowsocks,OPENVPN,52
1,germany.vpn,wireguard,OPENVPN,87
2,com.zoogvpn.android,xray,IKEV2,162
3,com.zoogvpn.android,xray,L2TP,156
4,com.zoogvpn.android,xray,PPTP,161
...,...,...,...,...
56,com.gaditek.purevpnics,openvpn,IKEV2,880
57,com.bitdefender.vpn,wireguard,IKEV2,2950
58,com.bitdefender.vpn,wireguard,OPENVPN,2990
59,com.bitdefender.vpn,wireguard,L2TP,45


In [ ]:
APP_A = "com.vpn99"
APP_B = "com.instabridge.android"

def load_app_df(app: str) -> pd.DataFrame:
    file_csv = f"{data_path}/{app}/{app}_attribution.csv"
    censys_csv = f"{out_path}/{app}/ip_vpn_protocols.csv"

    df_attr = pd.read_csv(file_csv)
    censys_df = pd.read_csv(censys_csv).rename(columns={"IP": "ip", "Protocols": "censys_protocols"})
    # left join so we don't drop attribution rows
    df = df_attr.merge(censys_df, on="ip", how="left")

    # normalize column existence
    if "protocols" not in df.columns:
        raise ValueError(f"{app}: missing 'protocols' column in attribution CSV")
    if "ip" not in df.columns:
        raise ValueError(f"{app}: missing 'ip' column in attribution CSV")

    return df

def ips_all(df: pd.DataFrame) -> set:
    return set(df["ip"].dropna().astype(str).str.strip())

def ips_detectable(df: pd.DataFrame) -> set:
    """
    IPs that have any Censys visibility (i.e., detectable).
    """
    return set(
        df.loc[~pd.isna(df["censys_protocols"]), "ip"]
          .astype(str)
          .str.strip()
    )


def report_overlap(app_a: str, app_b: str):
    df_a = load_app_df(app_a)
    df_b = load_app_df(app_b)

    # ---- All attributed IPs ----
    a_all = ips_all(df_a)
    b_all = ips_all(df_b)
    inter_all = a_all & b_all

    # ---- Detectable IPs (Censys-visible) ----
    a_det = ips_detectable(df_a)
    b_det = ips_detectable(df_b)

    # ---- Detectable overlap on BOTH apps ----
    inter_detectable = inter_all & a_det & b_det

    def pct(num, den):
        return 0.0 if den == 0 else (100.0 * num / den)

    print("\n================ Overlap Report ================")
    print(f"A = {app_a}")
    print(f"B = {app_b}")

    print("\n--- Attribution overlap ---")
    print(f"{app_a} unique IPs: {len(a_all)}")
    print(f"{app_b} unique IPs: {len(b_all)}")
    print(f"Intersection: {len(inter_all)}")
    print(f"% of {app_a} present in {app_b}: {pct(len(inter_all), len(a_all)):.2f}%")
    print(f"% of {app_b} present in {app_a}: {pct(len(inter_all), len(b_all)):.2f}%")

    print("\n--- Detectable overlap (Censys-visible on BOTH apps) ---")
    print(f"{app_a} detectable IPs: {len(a_det)}")
    print(f"{app_b} detectable IPs: {len(b_det)}")
    print(f"Detectable intersection: {len(inter_detectable)}")
    print(f"% of {app_a} overlap that is detectable: {pct(len(inter_detectable), len(inter_all)):.2f}%")
    print(f"% of {app_a} detectable servers shared with {app_b}: {pct(len(inter_detectable), len(a_det)):.2f}%")

# Run it
report_overlap(APP_A, APP_B)


================ Overlap Report ================
A = com.vpn99
B = com.instabridge.android

--- Attribution overlap ---
com.vpn99 unique IPs: 1760
com.instabridge.android unique IPs: 362
Intersection: 336
% of com.vpn99 present in com.instabridge.android: 19.09%
% of com.instabridge.android present in com.vpn99: 92.82%

--- Detectable overlap (Censys-visible on BOTH apps) ---
com.vpn99 detectable IPs: 345
com.instabridge.android detectable IPs: 361
Detectable intersection: 335
% of com.vpn99 overlap that is detectable: 99.70%
% of com.vpn99 detectable servers shared with com.instabridge.android: 97.10%
